In [15]:
import requests as rq
from bs4 import BeautifulSoup

url = 'https://finance.naver.com/sise/sise_deposit.naver'
data = rq.get(url)
data_html = BeautifulSoup(data.content)
parse_day = data_html.select_one('div.subtop_sise_graph2 > ul.subtop_chart_note > li > span.tah').text

print(parse_day)

  |  2023.06.07


In [21]:
import re

biz_day = re.findall('[0-9]+', parse_day)
biz_day = ''.join(biz_day)

print(biz_day)
# print(type(biz_day))
# biz_day = '20230609'

20230607
<class 'str'>


In [24]:
import requests as rq
from io import BytesIO
import pandas as pd

gen_otp_url = 'http://data.krx.co.kr/comm/fileDn/GenerateOTP/generate.cmd'
gen_otp_stk = {
    'mktId' : 'STK',
    'trdDd' : biz_day,
    'money' : '1',
    'csvxls_isNo' : 'false',
    'name' : 'fileDown',
    'url' : 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
headers = {'Referer' : 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader'}
otp_stk = rq.post(gen_otp_url, gen_otp_stk, headers=headers).text

print(otp_stk)

/IXFE7MDRlHUt2LJJQBUSj8Pih6ndnmr4jQBRgR+ahURtSksuLS7Bnxpl86F7dAOkunw9BBwugQaSjGAcH15eakyjuVhvrgG1GQhF5BAeBYtBgM+EFJCxYg3zco1gIgRZqIo4cIzoURnTI8+MmkJ4m8vFLhSKmM794gFu+ThsO31lY4woqehX8j6OlXFDcfHdV4NbYo4+D2Rwcfj24VnU3Zpq3ik/Dyw3FdyOXhJkBI=


In [25]:
down_url = 'http://data.krx.co.kr/comm/fileDn/download_csv/download.cmd'
down_sector_stk = rq.post(down_url, {'code' : otp_stk}, headers=headers)
sector_stk = pd.read_csv(BytesIO(down_sector_stk.content), encoding='EUC-KR')

sector_stk.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,095570,AJ네트웍스,KOSPI,서비스업,4415,35,0.80,206720432425
1,006840,AK홀딩스,KOSPI,기타금융,18740,-70,-0.37,248259293140
2,027410,BGF,KOSPI,기타금융,4140,15,0.36,396267514740
3,282330,BGF리테일,KOSPI,유통업,186100,2600,1.42,3216534906600
4,138930,BNK금융지주,KOSPI,기타금융,7010,70,1.01,2284806074460


In [27]:
gen_otp_ksq = {
    'mktId' : 'KSQ',
    'trdDd' : biz_day,
    'money' : '1',
    'csvxls_isNo' : 'false',
    'name' : 'fileDown',
    'url' : 'dbms/MDC/STAT/standard/MDCSTAT03901'
}
otp_ksq = rq.post(gen_otp_url, gen_otp_ksq, headers=headers).text

down_sector_ksq = rq.post(down_url, {'code' : otp_ksq}, headers=headers)
sector_ksq = pd.read_csv(BytesIO(down_sector_ksq.content), encoding = 'EUC-KR')

sector_ksq.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액
0,060310,3S,KOSDAQ,기계·장비,2965,5,0.17,143911143530
1,054620,APS,KOSDAQ,금융,11050,310,2.89,225356142050
2,265520,AP시스템,KOSDAQ,반도체,21550,150,0.70,329314622550
3,211270,AP위성,KOSDAQ,통신장비,14260,170,1.21,215073655040
4,126600,BGF에코머티리얼즈,KOSDAQ,화학,6890,-20,-0.29,281587506460


In [28]:
krx_sector = pd.concat([sector_stk, sector_ksq]).reset_index(drop=True)
krx_sector['종목명'] = krx_sector['종목명'].str.strip()
krx_sector['기준일'] = biz_day

krx_sector.head()

,종목코드,종목명,시장구분,업종명,종가,대비,등락률,시가총액,기준일
0,095570,AJ네트웍스,KOSPI,서비스업,4415,35,0.80,206720432425,20230609
1,006840,AK홀딩스,KOSPI,기타금융,18740,-70,-0.37,248259293140,20230609
2,027410,BGF,KOSPI,기타금융,4140,15,0.36,396267514740,20230609
3,282330,BGF리테일,KOSPI,유통업,186100,2600,1.42,3216534906600,20230609
4,138930,BNK금융지주,KOSPI,기타금융,7010,70,1.01,2284806074460,20230609
